In [2]:
# Inserting libraries to work
import pandas as pd
import numpy as np
from pathlib import Path
import os
import re
import time
import comtradeapicall
import glob
import traceback
import pyarrow
import fastparquet

In [ ]:
# insert relevant two digit codes database
two_dig = pd.read_csv("relevant_two_digit_codes.csv")

# insert excel database to get the 4 digit codes
hscodes = pd.read_excel("HSCodeandDescription.xlsx")
hscodes["Code"] = hscodes["Code"].astype(str).str.strip()

#  Ensure two_dig is a list of 2-digit strings like ["01","27","84",...]
if isinstance(two_dig, (pd.DataFrame, pd.Series)):
    # If it's a DataFrame with a column named "item2" or "Code", pick the right one:
    col = "item2" if "item2" in getattr(two_dig, "columns", []) else two_dig.name
    two_dig_list = two_dig[col].astype(str).str.zfill(2).tolist() if isinstance(two_dig, pd.DataFrame) else two_dig.astype(str).str.zfill(2).tolist()
else:
    two_dig_list = [str(x).zfill(2) for x in two_dig]

two_dig_set = set(two_dig_list)
print(two_dig_set)

# Filter HS codes to get relevant 4-digit codes
hs_level4_relevant = hscodes[
    (hscodes["Level"] == 4) &
    (hscodes["Code"].str[:2].isin(two_dig_set))
].copy()

hs_level4_relevant.head()

hs_four_code = hs_level4_relevant["Code"].tolist()
hs_four_code


{'87', '39', '72', '30', '85', '95', '29', '70', '91', '20', '90', '27', '25', '80', '51', '84', '50', '81', '10', '40', '14'}


['1001',
 '1002',
 '1003',
 '1004',
 '1005',
 '1006',
 '1007',
 '1008',
 '1401',
 '1404',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2501',
 '2502',
 '2503',
 '2504',
 '2505',
 '2506',
 '2507',
 '2508',
 '2509',
 '2510',
 '2511',
 '2512',
 '2513',
 '2514',
 '2515',
 '2516',
 '2517',
 '2518',
 '2519',
 '2520',
 '2521',
 '2522',
 '2523',
 '2524',
 '2525',
 '2526',
 '2528',
 '2529',
 '2530',
 '2701',
 '2702',
 '2703',
 '2704',
 '2705',
 '2706',
 '2707',
 '2708',
 '2709',
 '2710',
 '2711',
 '2712',
 '2713',
 '2714',
 '2715',
 '2716',
 '2901',
 '2902',
 '2903',
 '2904',
 '2905',
 '2906',
 '2907',
 '2908',
 '2909',
 '2910',
 '2911',
 '2912',
 '2913',
 '2914',
 '2915',
 '2916',
 '2917',
 '2918',
 '2919',
 '2920',
 '2921',
 '2922',
 '2923',
 '2924',
 '2925',
 '2926',
 '2927',
 '2928',
 '2929',
 '2930',
 '2931',
 '2932',
 '2933',
 '2934',
 '2935',
 '2936',
 '2937',
 '2938',
 '2939',
 '2940',
 '2941',
 '2942',
 '3001',
 '3002',
 '3003',
 '3004',
 '3005',
 

In [9]:
subscription_key = "3052290edec74a228fdf50950851da90"

def get_comtrade_hs4_official(
    subscription_key: str,
    year: int,
    reporter_code: str,
    hs_four_code: list[str],            # <-- your ready-made HS4 list
    partner_code: str | None = None,    # None = all partners
    flow_code: str = "X",
    cl_code: str = "HS",
    breakdown_mode: str = "classic",
    max_records: int = 90000,
    chunk_size: int = 50
):
    out = []

    for i in range(0, len(hs_four_code), chunk_size):
        chunk = hs_four_code[i:i + chunk_size]
        cmd_code = ",".join(chunk)

        df_chunk = comtradeapicall.getFinalData(
            subscription_key,
            typeCode="C",
            freqCode="A",
            clCode=cl_code,
            period=str(year),
            reporterCode=str(reporter_code),
            cmdCode=cmd_code,                 # <-- ONLY these HS4 codes
            flowCode=flow_code,
            partnerCode=partner_code,
            partner2Code=None,
            customsCode=None,
            motCode=None,
            maxRecords=max_records,
            format_output="JSON",
            aggregateBy=None,
            breakdownMode=breakdown_mode,
            countOnly=None,
            includeDesc=True
        )

        if df_chunk is not None and len(df_chunk) > 0:
            out.append(df_chunk)

    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

In [12]:
# --- inputs ---
#years = [2003, 2009, 2017, 2021]
years = [2009]
# due to quotas, given the account, the free account allows to get all the data for one year per day
# year 2003 done
# year 2009 
# year 2017
# year 2021 


reporter_codes = ["32","76","222","251","276",
                  "340","380","392","484","554",
                  "566","608","643","682","724",
                  "740","792","842","716","156","124","826",
                  "702"]
flow_codes = ["X", "M"]
partner_code = None
out_dir = "comtrade_data_hs4"
sleep_s = 5.0

os.makedirs(out_dir, exist_ok=True)

keep_cols = ["flowCode", "reporterCode", "reporterISO", "partnerCode", "partnerISO",
             "cmdCode", "cmdDesc", "qtyUnitAbbr", "qty", "fobvalue"]

def safe_filename(s: str) -> str:
    s = str(s).strip()
    return re.sub(r"[^\w\-\.]+", "_", s)

# IMPORTANT: you already have this list prepared:
# hs_four_code = hs_level4_relevant["Code"].tolist()

for year in years:
    for reporter_code in reporter_codes:
        for flow_code in flow_codes:
            try:
                df_hs4 = get_comtrade_hs4_official(
                    subscription_key=subscription_key,
                    year=year,
                    reporter_code=str(reporter_code),
                    hs_four_code=hs_four_code,     # <-- HS4 framework
                    partner_code=partner_code,
                    flow_code=str(flow_code),
                    chunk_size=50                  # adjust if needed
                )

                if df_hs4 is None or df_hs4.empty:
                    print(f"⚠ Empty result: reporter={reporter_code}, flow={flow_code}, year={year}")
                    continue

                cols_present = [c for c in keep_cols if c in df_hs4.columns]
                df_hs4_boni = df_hs4[cols_present].copy()

                rep_iso = df_hs4_boni["reporterISO"].iloc[0] if "reporterISO" in df_hs4_boni.columns else reporter_code
                flw = df_hs4_boni["flowCode"].iloc[0] if "flowCode" in df_hs4_boni.columns else flow_code

                fname = f"{safe_filename(rep_iso)}_{safe_filename(flw)}_{year}_HS4.csv"
                fpath = os.path.join(out_dir, fname)

                df_hs4_boni.to_csv(fpath, index=False)
                print(f"✓ Saved {fpath}  ({df_hs4_boni.shape[0]} rows, {df_hs4_boni.shape[1]} cols)")

                time.sleep(sleep_s)

            except Exception as e:
                print(f"✗ Failed: reporter={reporter_code}, flow={flow_code}, year={year} → {e}")

✓ Saved comtrade_data_hs4/ARG_X_2009_HS4.csv  (9972 rows, 10 cols)
✓ Saved comtrade_data_hs4/ARG_M_2009_HS4.csv  (10890 rows, 10 cols)
✓ Saved comtrade_data_hs4/BRA_X_2009_HS4.csv  (18196 rows, 10 cols)
✓ Saved comtrade_data_hs4/BRA_M_2009_HS4.csv  (13205 rows, 10 cols)
✓ Saved comtrade_data_hs4/SLV_X_2009_HS4.csv  (2348 rows, 10 cols)
✓ Saved comtrade_data_hs4/SLV_M_2009_HS4.csv  (7894 rows, 10 cols)
✓ Saved comtrade_data_hs4/FRA_X_2009_HS4.csv  (36896 rows, 10 cols)
✓ Saved comtrade_data_hs4/FRA_M_2009_HS4.csv  (19504 rows, 10 cols)
✓ Saved comtrade_data_hs4/DEU_X_2009_HS4.csv  (51224 rows, 10 cols)
✓ Saved comtrade_data_hs4/DEU_M_2009_HS4.csv  (24624 rows, 10 cols)
✓ Saved comtrade_data_hs4/HND_X_2009_HS4.csv  (2243 rows, 10 cols)
✓ Saved comtrade_data_hs4/HND_M_2009_HS4.csv  (6843 rows, 10 cols)
✓ Saved comtrade_data_hs4/ITA_X_2009_HS4.csv  (38926 rows, 10 cols)
✓ Saved comtrade_data_hs4/ITA_M_2009_HS4.csv  (18201 rows, 10 cols)
✓ Saved comtrade_data_hs4/JPN_X_2009_HS4.csv  (25145 